In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.set_option('display.max_columns', None) 

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df = df.drop(columns=['Ended', 'Explore link'])
df['Started'] = pd.to_datetime(df['Started'])

In [4]:
# Function to convert string to numeric
def convert_to_number(s):
    s = s.replace('+', '')  # Remove the '+' sign
    if 'M' in s:
        return int(float(s.replace('M', '')) * 1_000_000)
    elif 'K' in s:
        return int(float(s.replace('K', '')) * 1_000)
    else:
        return int(s)  # For plain numbers

# Apply the function to the array
numeric_data = np.array([convert_to_number(value) for value in df['Search volume']])

# Output the result
df['Search volume'] = numeric_data

In [5]:
df.head()

,Trends,Search volume,Started,Trend breakdown
0,wicked book,2000000,2024-11-20 22:00:00+05:00,"wicked book,elphaba,wicked musical,wicked trai..."
1,moana 2,1000000,2024-11-22 07:20:00+05:00,"moana 2,moana 2 release date,when does moana 2..."
2,who won dancing with the stars 2024,1000000,2024-11-25 22:00:00+05:00,"who won dancing with the stars 2024,dancing wi..."
3,pam bondi,1000000,2024-11-21 13:40:00+05:00,pam bondi
4,lake effect snow warning,1000000,2024-11-27 12:20:00+05:00,"lake effect snow warning,thanksgiving,thanksgi..."


In [6]:
from transformers import pipeline

# Use a zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [7]:
categories = [
    "Technology and Science",
    "Health and Wellness",
    "Travel and Leisure",
    "Food and Lifestyle",
    "Education and Knowledge",
    "Sports and Recreation",
    "Finance and Business",
    "Arts and Entertainment",
    "Relationships and Society",
    "Environment and Current Events"
]

In [8]:
#setting empty values for the columns
index = -1
for label in categories:
    df[label] = -1

for j in range(len(df)):
    #counter for progress/debugging
    index+=1
    if(index%10 == 0): 
        print(index)
        
    #running the classifier on the column    
    res = classifier(
        df.iloc[j]['Trends'],
        candidate_labels = categories,
        multi_label = True
    )
    #setting the column values according to the output from the classifier ("_m" = multiclass)
    for i in range(len(res['labels'])):
        df[res['labels'][i]].iloc[j] = res['scores'][i]

0
10
20
30
40
50
60
70
80
90
100
110
120
130


In [9]:
# df_demo['max'] = df_demo[['cost_m', 'efficient_m', 'effective_m', 'ease of use_m']].max(axis=1)

In [10]:
df['max'] = df.iloc[:,4:].max(axis=1)

In [11]:
len(df)
len(df[df['max'] > 0.8])

138

27

In [12]:
df.to_csv('pretrained.csv')